In [ ]:
import requests
import json
from datetime import datetime, timedelta

# === MES CLÉS (après inscription) ===
CLIENT_ID = "confidentiel"
CLIENT_SECRET = "confidentiel"

# Étape 1 : Obtiens le token (auth)
def get_access_token():
    url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    payload = {
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET
    }
    response = requests.post(url, headers=headers, data=payload)
    if response.status_code == 200:
        token = response.json()["access_token"]
        print(f"Token obtenu ! Valable jusqu'à {datetime.now() + timedelta(minutes=30)}")
        return token
    else:
        print("Erreur auth :", response.text)
        return None

token = get_access_token()
headers = {"Authorization": f"Bearer {token}"}

Token obtenu ! Valable jusqu'à 2025-11-22 00:49:21.906895


In [2]:
import pandas as pd
import time
from datetime import datetime, timedelta
import requests

# === MES CLÉS (après inscription) ===
CLIENT_ID = "confidentiel"
CLIENT_SECRET = "confidentiel"

# Étape 1 : j'obtiens le token (auth)
def get_access_token():
    url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    auth_headers = {"Content-Type": "application/x-www-form-urlencoded"}
    payload = {
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET
    }
    response = requests.post(url, headers=auth_headers, data=payload)
    if response.status_code == 200:
        token = response.json()["access_token"]
        print(f"Token obtenu ! Valable jusqu'à {datetime.now() + timedelta(minutes=30)}")
        return token
    else:
        print("Erreur auth :", response.text)
        return None

# Get token and define headers for API calls
token = get_access_token()
headers = {"Authorization": f"Bearer {token}"}

# Tes 10 destinations
destinations = ["ACC", "LOS", "DKR", "CKY", "BKO", "NIM", "OUA", "LFW", "BRZ", "LBV"]

# Dates : +15, +30, +60, +90 jours (adapte si besoin)
dates_to_check = []
for i in [15, 30, 60, 90]:
    date = (datetime.now() + timedelta(days=i)).strftime("%Y-%m-%d")
    dates_to_check.append(date)

results = []

for dest in destinations:
    for dep_date in dates_to_check:
        print(f"Recherche ABJ → {dest} le {dep_date}...")

        # Endpoint Flight Offers Search
        url = "https://test.api.amadeus.com/v2/shopping/flight-offers"
        params = {
            "originLocationCode": "ABJ",
            "destinationLocationCode": dest,
            "departureDate": dep_date,
            "adults": 1,
            "travelClass": "ECONOMY",
            "nonStop": "false",  # Inclut directs et avec escale
            "max": 10  # Top 10 offres pour limiter
        }

        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()["data"]
            for offer in data:
                # Extrait prix total et compagnie
                price_total = float(offer["price"]["total"])
                currency = offer["price"]["currency"]
                airline = offer["itineraries"][0]["segments"][0]["carrierCode"]  # Première compagnie

                # Filtre sur tes 7 compagnies (ajoute si tu veux plus strict)
                target_airlines = ["HC", "ET", "KP", "BO", "Arik", "KQ", "WB"]  # Codes : HC=Air Côte, ET=Ethiopian, etc.
                if airline in target_airlines:
                    results.append({
                        "Date recherche": datetime.now().strftime("%Y-%m-%d"),
                        "Route": f"ABJ-{dest}",
                        "Date vol": dep_date,
                        "Compagnie": airline,  # Ajoute nom complet si besoin via un dict
                        "Prix total": price_total,
                        "Devise": currency,
                        "Durée": offer["itineraries"][0]["duration"]  # Bonus
                    })
        else:
            print(f"Erreur pour {dest}-{dep_date} :", response.status_code)

        # Pause pour rate limit (1 req/sec gratuit)
        time.sleep(1)

# Export Excel
df = pd.DataFrame(results)
df = df.sort_values(["Route", "Date vol", "Prix total"])
df.to_excel(f"Veille_tarifaire_Amadeus_ABJ_{datetime.now().strftime('%Y%m%d')}.xlsx", index=False)
print(f"Terminé ! {len(results)} offres collectées. Fichier Excel généré.")

Token obtenu ! Valable jusqu'à 2025-11-23 00:40:17.820547
Recherche ABJ → ACC le 2025-12-08...
Recherche ABJ → ACC le 2025-12-23...
Recherche ABJ → ACC le 2026-01-22...
Recherche ABJ → ACC le 2026-02-21...
Recherche ABJ → LOS le 2025-12-08...
Recherche ABJ → LOS le 2025-12-23...
Recherche ABJ → LOS le 2026-01-22...
Recherche ABJ → LOS le 2026-02-21...
Recherche ABJ → DKR le 2025-12-08...
Recherche ABJ → DKR le 2025-12-23...
Recherche ABJ → DKR le 2026-01-22...
Recherche ABJ → DKR le 2026-02-21...
Recherche ABJ → CKY le 2025-12-08...
Recherche ABJ → CKY le 2025-12-23...
Recherche ABJ → CKY le 2026-01-22...
Recherche ABJ → CKY le 2026-02-21...
Recherche ABJ → BKO le 2025-12-08...
Recherche ABJ → BKO le 2025-12-23...
Recherche ABJ → BKO le 2026-01-22...
Recherche ABJ → BKO le 2026-02-21...
Recherche ABJ → NIM le 2025-12-08...
Recherche ABJ → NIM le 2025-12-23...
Recherche ABJ → NIM le 2026-01-22...
Recherche ABJ → NIM le 2026-02-21...
Recherche ABJ → OUA le 2025-12-08...
Recherche ABJ → O